# Following Along Youtube Tutorial

### We will be using the Keras Tuner package to tune a NN

In [ ]:
# We will use Fashion Mnist Dataset.

from tensorflow.keras.datasets import fashion_mnist
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, Activation, Flatten, MaxPooling2D
import time

# The tuner parts

from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters



In [ ]:
# Time 

Log_Dir = f"{int(time.time())}"

In [ ]:
# We want to split into train and test. 

(x_train,y_train), (x_test,y_test) = fashion_mnist.load_data()
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

In [ ]:
# Building our model.


def build_model(hp):
    # Build our model
    model = keras.models.Sequential()
    
    # Conv Layer
    model.add(Conv2D(hp.Int('Input units',32,256,32), (3,3), input_shape = x_train.shape[1:]))
    
    # Activation layer
    model.add(Activation('relu'))
        
    # Pooling Layer
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    for i in range(hp.Int('n_layers',1,4)):
        # Conv Layer
        model.add(Conv2D(hp.Int('Units',32,256,32),(3,3)))
        
        # Activation layer
        model.add(Activation('relu'))
    
    # Flatten 
    model.add(Flatten())
    
    # Dense Layer
    model.add(Dense(10))
    
    # Finally we have a softmax for the categories 
    model.add(Activation('softmax'))
    
    # Compile the model
    model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [ ]:
# Setting up the Tuner 

tuner = RandomSearch(
    build_model, 
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial = 1,
    directory = Log_Dir
)

# Tuner Search here. 

tuner.search(x = x_train, y = y_train, epochs = 1, batch_size = 64, validation_data = (x_test,y_test))